In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm
import xgboost as xgb

import pickle

/Users/light/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
data_path = '../data/'
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
songs = pd.read_csv(data_path + 'songs1.csv')
members = pd.read_csv(data_path + 'members.csv')

song_cols = ['song_id', 'song_length', 'genre_ids', 'artist_name', 'composer',\
            'lyricist', 'language', 'song_name', 'song_country', 'song_year']

train = train.merge(songs[song_cols], on='song_id', how='left')
test = test.merge(songs[song_cols], on='song_id', how='left')

members['registration_year'] = members['registration_init_time'].apply(lambda x: int(str(x)[0:4]))
members['registration_month'] = members['registration_init_time'].apply(lambda x: int(str(x)[4:6]))
members['registration_date'] = members['registration_init_time'].apply(lambda x: int(str(x)[6:8]))

members['expiration_year'] = members['expiration_date'].apply(lambda x: int(str(x)[0:4]))
members['expiration_month'] = members['expiration_date'].apply(lambda x: int(str(x)[4:6]))
members['expiration_date'] = members['expiration_date'].apply(lambda x: int(str(x)[6:8]))
member = members.drop(['registration_year', 'expiration_year'], axis=1)

members_cols = members.columns
train = train.merge(members[members_cols], on='msno', how='left')
test = test.merge(members[members_cols], on='msno', how='left')

train = train.fillna(-1)
test = test.fillna(-1)

In [13]:
for x in list(songs):
    print (x)

id
song_id
song_length
genre_ids
artist_name
composer
lyricist
language
song_name
isrc
song_country
song_year


In [15]:
train['artist_name'][:4]

0           Bastille
1    Various Artists
2                Nas
3           Soundway
Name: artist_name, dtype: object

In [3]:
#<class 'pandas.core.series.Series'> object
#pandas.core.frame.DataFrame

In [17]:
# Preprocess dataset
cols = list(train.columns)
cols.remove('target')

for col in tqdm(cols):
    try:
        if train[col].dtype == 'object':
            train[col] = train[col].apply(str)
            test[col] = test[col].apply(str)

            le = LabelEncoder()
            train_vals = list(train[col].unique())
            test_vals = list(test[col].unique())
            le.fit(train_vals + test_vals)
            train[col] = le.transform(train[col])
            test[col] = le.transform(test[col])

            print(col + ': ' + str(len(train_vals)) + ', ' + str(len(test_vals)))
    except:
        print (col, type(train[col]))
        break




  0%|          | 0/25 [00:00<?, ?it/s]
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/light/anaconda/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/Users/light/anaconda/lib/python3.5/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/light/anaconda/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

  4%|▍         | 1/25 [00:24<09:56, 24.84s/it]

msno: 30755, 25131


  8%|▊         | 2/25 [01:22<15:50, 41.33s/it]

song_id: 359966, 224753


 12%|█▏        | 3/25 [01:40<12:19, 33.61s/it]

...

song_name: 234112, 154708


 52%|█████▏    | 13/25 [06:44<06:13, 31.09s/it]

song_country: 110, 97


100%|██████████| 25/25 [06:57<00:00, 16.68s/it]

gender: 3, 3


In [18]:
ssss

NameError: name 'ssss' is not defined

In [19]:
print(train.head())
print(test.head())

X = np.array(train.drop(['target'], axis=1))
y = train['target'].values

X_test = np.array(test.drop(['id'], axis=1))
ids = test['id'].values

X_train, X_valid, y_train, y_valid = train_test_split(X, y, \
    test_size=0.2, random_state=0)


d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_valid, label=y_valid) 
d_test = xgb.DMatrix(X_test)

eval_set = [(X_train, y_train), (X_valid, y_valid)]
watchlist = [(d_train, 'train'), (d_valid, 'valid')]

    msno  song_id  source_system_tab  source_screen_name  source_type  target  \
0   9176    86884                  2                   8            7       1   
1  19273   260594                  4                   9            5       1   
2  19273   140755                  4                   9            5       1   
3  19273    27577                  4                   9            5       1   
4   9176    38706                  2                   8            7       1   

   song_length  genre_ids  artist_name  composer        ...         bd  \
0     206471.0        308         3785     16654        ...          0   
1     284584.0         98        36868        71        ...         24   
2     225396.0         98        24602     51541        ...         24   
3     255512.0          7        31652     41992        ...         24   
4     187802.0          3         5191      9702        ...          0   

   gender  registered_via  registration_init_time  expiration_date  

In [ ]:
for x in list(train)
    print (x)

In [20]:
# Train model, evaluate and make predictions
def model(eta, max_depth):
    params = {}
    params['objective'] = 'binary:logistic'
    params['eta'] = eta #0.7
    params['max_depth'] = max_depth #5
    params['silent'] = 1
    params['eval_metric'] = 'auc'
    params['subsample'] = 1.0 #1.0
    params['min_child_weight'] = 5 #5
    params['colsample_bytree'] = 0.2 # 0.2
    evals_result = {}
    
    model = xgb.train(params, d_train, 105, watchlist, early_stopping_rounds=20, \
        maximize=True, verbose_eval=10, evals_result = evals_result)

    p_test = model.predict(d_test)
    return [p_test, [evals_result['train']['auc'][-1], evals_result['valid']['auc'][-1]]]

The history saving thread hit an unexpected error (OperationalError('unable to open database file',)).History will not be written to the database.


In [58]:
etas = [0.01 * (2 ** x) for x in range(1, 8)] # 0.01 to 1.28
subsamples = [0.1 * x for x in range(5, 11)]
min_child_weights = [ x for x in range(2, 10)] 
colsample_bytrees = [0.1 * x for x in range(1, 11)]
p_tests = []
weights = []

In [ ]:
for max_depth in range(7,15):
    for eta in [0.9]:
        p_test, res = model(eta, max_depth)
        with open('%s_%s.pickle' % (eta*10, max_depth),\
                                      'wb')as handle:
            pickle.dump(p_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open('%s_%s_res.pickle' % (eta*10, max_depth),\
                                      'wb')as handle:
            pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)
        p_tests += p_test,
        weights += res,

[0]	train-auc:0.656992	valid-auc:0.656627
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 20 rounds.
[10]	train-auc:0.685999	valid-auc:0.684967
[20]	train-auc:0.696548	valid-auc:0.69481
[30]	train-auc:0.70385	valid-auc:0.701774
[40]	train-auc:0.708127	valid-auc:0.705447
[50]	train-auc:0.713368	valid-auc:0.710174
[60]	train-auc:0.719463	valid-auc:0.715828
[70]	train-auc:0.72356	valid-auc:0.719519
[80]	train-auc:0.726944	valid-auc:0.722293
[90]	train-auc:0.73067	valid-auc:0.725706
[100]	train-auc:0.733022	valid-auc:0.727608
[0]	train-auc:0.659704	valid-auc:0.659181
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 20 rounds.
[10]	train-auc:0.691551	valid-auc:0.689757
[20]	train-auc:0.703835	valid-auc:0.700952
[30]	train-auc:0.714268	valid-auc:0.710014
[40]	train-auc:0.720057	valid-auc:0.714803
[50]	train-auc:0.725903	valid-a

In [60]:
weights = [x[1] for x in weights]
print (weights)

[0.726066, 0.737759, 0.748974, 0.76023, 0.768716, 0.774832, 0.780295, 0.783225]


[array([ 0.61823994,  0.68429184,  0.15756422, ...,  0.46034351,
        0.28037271,  0.52262759], dtype=float32), array([ 0.64945638,  0.72622496,  0.08256978, ...,  0.40195748,
        0.56350332,  0.38161099], dtype=float32), array([ 0.57295883,  0.66559821,  0.16085353, ...,  0.34414583,
        0.2465423 ,  0.32933512], dtype=float32)]


In [61]:
total = np.sum(weights)
print (total)
result = (np.array(p_tests).T * np.array(weights)).T

6.080097


In [62]:
for x in result[0]:
    print (x)
    break

0.367186068286


In [63]:
print (np.shape(result))
mat = np.sum(np.matrix(result), axis = 0)
p_test = [x / total for x in (mat.tolist()[0])]
print (np.shape(p_test))

(8, 2556790)
(2556790,)


In [64]:
for x in p_test[:10]:
    print (x)

0.570655484057
0.581413742763
0.149166142116
0.1587308109
0.092280751841
0.082292448482
0.0999132944131
0.60623654552
0.116966316641
0.75354827326


In [65]:
print (np.sum(p_test), total)
print(p_test[:30])

1153547.80067 6.080097
[0.5706554840566217, 0.581413742762942, 0.14916614211646084, 0.15873081090045421, 0.092280751840975533, 0.082292448482018735, 0.099913294413103482, 0.60623654551980322, 0.11696631664126375, 0.75354827325956553, 0.85834640936327078, 0.21649286939877421, 0.21201662564667348, 0.35645880938272151, 0.47542811376319372, 0.27627696581163907, 0.31095170748881729, 0.360451222954489, 0.41398138809861362, 0.75623951222149066, 0.83159836347863036, 0.53081967950078768, 0.60549767424237733, 0.65977437116158433, 0.47550732270315899, 0.47470735329351438, 0.49624855884680857, 0.48989081351051711, 0.48362556074529206, 0.39779905907358626]


In [ ]:
# with open('p_tests.pickle', 'wb') as handle:
#     pickle.dump(p_tests, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('weights.pickle', 'wb') as handle:
#     pickle.dump(weights, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [56]:
print (np.sum(p_test))

1169748.15508


In [66]:
# Prepare submission
subm = pd.DataFrame()
subm['id'] = ids
subm['target'] = p_test
print(len(ids), len(p_test))
subm.to_csv('../submissions/submission.csv', index=False)

2556790 2556790
